In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\vikra\Instacart Basket Analysis 06_24'

In [6]:
df_ords = pd.read_csv(r'C:\Users\vikra\Instacart Basket Analysis 06_24\Data\Original Data\orders.csv', index_col = False )
df_prods = pd.read_csv( r'C:\Users\vikra\Instacart Basket Analysis 06_24\Data\Original Data\products.csv', index_col = False)
df_dep = pd.read_csv( r'C:\Users\vikra\Instacart Basket Analysis 06_24\Data\Original Data\departments.csv', index_col = False)

# 2.consistency checks

In [24]:

df_ords.describe()


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


# 3.Check for mixed-type data & 4. Fixing the data types.

In [19]:
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)
      

#  It seems there is no column with mix type data in orders data. we will check with product data.

In [18]:
for col in df_prods.columns.tolist():
  weird = (df_prods[[col]].map(type) != df_prods[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_prods[weird]) > 0:
    print (col)

product_name


In [21]:
df_prods['product_name'] = df_prods['product_name'].astype('str')


In [22]:
df_prods.dtypes

product_id         int64
product_name      object
aisle_id           int64
department_id      int64
prices           float64
dtype: object

The data type of product name has been successfully changed.

# 5.Check for missing values in df_ords 
# 6.Address the missing values

In [25]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

Only column with missing values is the "days_since_prior_order" column, and it’s missing 206209 values.

Address the missing values

In [30]:
df_ords_nan = df_ords[ df_ords['days_since_prior_order'].isnull() == True]

In [31]:
df_ords_nan

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
11,2168274,2,prior,1,2,11,NaN
26,1374495,3,prior,1,1,14,NaN
39,3343014,4,prior,1,6,11,NaN
45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...
3420930,969311,206205,prior,1,4,12,NaN
3420934,3189322,206206,prior,1,3,18,NaN
3421002,2166133,206207,prior,1,6,19,NaN
3421019,2227043,206208,prior,1,1,15,NaN


In [48]:
df_ords.shape

(3421083, 7)

In [60]:
df_ords_clean = df_ords[df_ords['days_since_prior_order'].isnull() == False]

In [61]:
df_ords_clean.shape

(3421083, 7)

In [52]:
df_ords['days_since_prior_order'].fillna(7, inplace= False)

0           7.0
1          15.0
2          21.0
3          29.0
4          28.0
           ... 
3421078    29.0
3421079    30.0
3421080    18.0
3421081     7.0
3421082    30.0
Name: days_since_prior_order, Length: 3421083, dtype: float64

In [53]:
df_ords.shape

(3421083, 7)

In [65]:
df_prods.isnull().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
prices           0
dtype: int64

# 7. Check for duplicate values &
# 8. Address the duplicates

In [66]:
df_dups_new = df_prods[df_prods.duplicated()]

In [67]:
df_dups_new

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [68]:
df_prods_no_dups = df_prods.drop_duplicates()

In [71]:
df_prods_no_dups.shape

(49688, 5)

In [49]:
df_dups = df_ords_clean[df_ords_clean.duplicated()]

In [50]:
df_dups

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order


In [74]:
df_ords.shape

(3421083, 7)

There are no duplicates in this set because it returned empty. however, if there were duplicates I would execute 
'.drop_duplicates()' to creat a new datafreame without duplicates.

In [76]:
df_ords.to_csv(os.path.join(path, 'Data','Prepared Data', 'orders_checked.csv'))
df_prods_no_dups.to_csv(os.path.join(path, 'Data','Prepared Data', 'products_checked.csv'))